In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pli
import seaborn as sns
import os
import tensorflow as tf
from PIL import Image
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import classification_report

In [ ]:
def clean_colab_memory(path):
    !rm -r {path}

def save_to_drive(src, dest):
    !cp -r {src} {dest}

# Esempio di utilizzo
# clean_colab_memory('/content/nih_chest_xray_subset')
# save_to_drive('/content/nih_chest_xray_subset', '/content/drive/My Drive/')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

labels_path = "/content/drive/MyDrive/nih_chest_xray_subset/sample_labels.csv"
labels_df = pd.read_csv(labels_path)

In [ ]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606 entries, 0 to 5605
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Image Index                  5606 non-null   object 
 1   Finding Labels               5606 non-null   object 
 2   Follow-up #                  5606 non-null   int64  
 3   Patient ID                   5606 non-null   int64  
 4   Patient Age                  5606 non-null   object 
 5   Patient Gender               5606 non-null   object 
 6   View Position                5606 non-null   object 
 7   OriginalImageWidth           5606 non-null   int64  
 8   OriginalImageHeight          5606 non-null   int64  
 9   OriginalImagePixelSpacing_x  5606 non-null   float64
 10  OriginalImagePixelSpacing_y  5606 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 481.9+ KB


In [ ]:
labels_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


In [ ]:
labels_df.shape

(5606, 11)

In [ ]:
labels_df['Finding Labels'].value_counts()

,count
Finding Labels,
No Finding,3044
Infiltration,503
Effusion,203
Atelectasis,192
Nodule,144
...,...
Atelectasis|Emphysema|Infiltration|Nodule|Pneumothorax,1
Cardiomegaly|Effusion|Mass|Pneumothorax,1
Cardiomegaly|Edema|Infiltration|Nodule,1


In [ ]:
data = []

for _, row in labels_df.iterrows():
    image_path = os.path.join('/content/drive/MyDrive/nih_chest_xray_subset/images', row['Image Index'])
    labels = row['Finding Labels'].split('|')
    data.append({'image_path': image_path, 'labels': labels})

print(data[:5])

[{'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000013_005.png', 'labels': ['Emphysema', 'Infiltration', 'Pleural_Thickening', 'Pneumothorax']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000013_026.png', 'labels': ['Cardiomegaly', 'Emphysema']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000017_001.png', 'labels': ['No Finding']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000030_001.png', 'labels': ['Atelectasis']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000032_001.png', 'labels': ['Cardiomegaly', 'Edema', 'Effusion']}]


In [ ]:
num_class_1 = sum(["Infiltration" in r["labels"] for r in data])
print(num_class_1)

967


In [ ]:
num_class_0 = sum(["Infiltration" not in r["labels"] for r in data])
print(num_class_0)

4639


In [ ]:
class_1_data = [r for r in data if "Infiltration" in r["labels"]]

print(f"Esempi della classe 1 disponibili per augmentation: {len(class_1_data)}")

Esempi della classe 1 disponibili per augmentation: 967


In [ ]:
def preprocess_item(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224]) / 255.0
    return image, label

paths = [item['image_path'] for item in data]
labels = [1 if "Infiltration" in item["labels"] else 0 for item in data]

# Preprocessing con parallelismo
dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
dataset = dataset.map(lambda x, y: preprocess_item(x, y), num_parallel_calls=tf.data.AUTOTUNE)

dataset_nobatch = dataset
dataset = dataset.cache()

In [ ]:
# Shuffle per migliorare il training
dataset = dataset.shuffle(buffer_size=1000)  # Cambia il buffer in base al dataset

# Creazione dei batch # Prefetch per caricare il batch successivo in parallelo
dataset = dataset.batch(64).prefetch(tf.data.AUTOTUNE)

In [ ]:
for images, labels in dataset.take(1):
    print(f"Batch shape: {images.shape}, Labels shape: {labels.shape}")

In [ ]:
# loaded_dataset = tf.data.Dataset.load("/content/drive/MyDrive/nih_chest_xray_subset/processed_dataset")

In [ ]:
#total_images = 0
#for images, labels in dataset:
#   total_images += images.shape[0]
#
#print(f"Total_number_of: {total_images}")

In [ ]:
# tf.data.experimental.save(dataset,"/content/drive/MyDrive/nih_chest_xray_subset/processed_dataset")

# TensorFlow Keras

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False # don't update base_model weights during training

model = tf.keras.Sequential([base_model, tf.keras.layers.GlobalMaxPooling2D(), tf.keras.layers.Dense(1, activation='sigmoid')])
# GlobalMaxPooling2D: Perde un po' di accuratezza complessiva, ma migliora nel riconoscimento della classe 1 rispetto a GlobalAveragePooling2D
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
dataset_size = len(dataset)

test_size = int(dataset_size * 0.2)
validation_size = int(dataset_size * 0.1)
train_size = dataset_size - test_size - validation_size

test_set = dataset.take(test_size)  # Primi 20% per il test
validation_set = dataset.skip(test_size).take(validation_size)  # Successivi 20% per validazione
train_set = dataset.skip(test_size + validation_size)  # Resto per il training

In [ ]:
print(f"Dataset totale (batch): {dataset_size}")
print(f"Test set (batch): {test_size}")
print(f"Validation set (batch): {validation_size}")
print(f"Training set (batch): {train_size}")

Dataset totale (batch): 88
Test set (batch): 17
Validation set (batch): 8
Training set (batch): 63


In [ ]:
history = model.fit(train_set, epochs=4, validation_data = validation_set)

Epoch 1/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 314s 4s/step - accuracy: 0.7111 - loss: 1.1601 - val_accuracy: 0.7910 - val_loss: 0.5263
Epoch 2/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 227s 4s/step - accuracy: 0.7718 - loss: 0.6076 - val_accuracy: 0.8672 - val_loss: 0.4378
Epoch 3/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 240s 3s/step - accuracy: 0.8093 - loss: 0.5498 - val_accuracy: 0.8008 - val_loss: 0.5933
Epoch 4/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 234s 4s/step - accuracy: 0.7854 - loss: 0.5485 - val_accuracy: 0.8320 - val_loss: 0.4488


In [ ]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

17/17 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.8047 - loss: 0.4865
Test Loss: 0.48007988929748535, Test Accuracy: 0.8125


In [ ]:
y_true = []  # Ground truth
y_pred = []  # Predizioni
for images, labels in test_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype(int).flatten())

print(classification_report(y_true, y_pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
              precision    recall  f1-score   support

           0       0.88      0.93      0.90       948
           1       0.22      0.14      0.17       140

    accuracy                           0.83      1088
   macro avg       0.55      0.53      0.54      1088
weighted avg       0.80      0.83      0.81      1088



This is due to the imbalance of the dataset:  
the model almost always predicts 0, largely ignoring class 1.  
On class 1 we experienced:  
Precision = 50%: The model makes few predictions like 1, but half of these predictions are correct.  
Recall = 2%: The model identifies only 2% of the real class 1 examples

# Pytorch

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CovNet(nn.Module):
  def __init__(self, channels, classes):
    super(CovNet, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=channels, out_channels=32, kernel_size=(5,5), padding='same')
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5,5), padding='same')
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

    self.flatten = nn.Flatten()

    self.fc1 = nn.Linear(in_features=64*56*56, out_features=128)
    self.relu3 = nn.ReLU()

    self.fc2 = nn.Linear(in_features=128, out_features=classes)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = nn.functional.relu(self.conv1(x))
    x = self.maxpool1(x)
    x = nn.functional.relu(self.conv2(x))
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = nn.functional.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.softmax(x)
    return x

In [ ]:
learning_rate = 0.001
num_epochs = 10
num_classes = 2
batch_size = 64
criterion = nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
model = CovNet(channels=3, classes=2).to(device)

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ChestXRayDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        """
        Args:
            image_paths (list): List of file paths to images.
            labels (list): List of labels corresponding to the images.
            transform (callable, optional): Transform to be applied on an image.
        """
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        # Apply transformations (if any)
        if self.transform:
            image = self.transform(image)

        # Get the label
        label = self.labels[idx]

        return image, label

# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Example CSV file containing image paths and labels
csv_file = '/content/drive/MyDrive/nih_chest_xray_subset/sample_labels.csv'  # Replace with your file path
labels_df = pd.read_csv(csv_file)

# Extract image paths and labels
data = []
labels = []  # Separate list for labels

for _, row in labels_df.iterrows():
    image_path = os.path.join('/content/drive/MyDrive/nih_chest_xray_subset/images', row['Image Index'])
    label = 1 if "Infiltration" in row['Finding Labels'].split('|') else 0  # Binary label
    data.append(image_path)  # Append only the image path
    labels.append(label)  # Append the binary label

# Split into train, test, and validation sets
train_size = int(0.7 * len(data))
test_size = int(0.2 * len(data))
val_size = len(data) - train_size - test_size

train_paths, train_labels = data[:train_size], labels[:train_size]
test_paths, test_labels = data[train_size:train_size+test_size], labels[train_size:train_size+test_size]
val_paths, val_labels = data[train_size+test_size:], labels[train_size+test_size:]


# Create datasets
train_dataset = ChestXRayDataset(train_paths, train_labels, transform=transform)
test_dataset = ChestXRayDataset(test_paths, test_labels, transform=transform)
val_dataset = ChestXRayDataset(val_paths, val_labels, transform=transform)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import time
import copy

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    import time
    import copy
    import torch

    since = time.time()

    test_acc_history = []
    test_loss_history = []
    train_acc_history = []
    train_loss_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Dynamically determine dataset size
            dataset_size = sum(1 for _ in dataloaders[phase])

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # Backward pass and optimization in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'test':
                test_acc_history.append(epoch_acc.cpu().numpy())
                test_loss_history.append(epoch_loss)
            if phase == 'train':
                train_acc_history.append(epoch_acc.cpu().numpy())
                train_loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    history_dict = {'train_loss': train_loss_history, 'train_accuracy': train_acc_history,
                    'test_loss': test_loss_history, 'test_accuracy': test_acc_history}
    return model, history_dict


In [ ]:
train_model(model,
            {'train':train_loader},
            criterion=criterion,
            optimizer=optimizer,
            num_epochs=5)

Epoch 1/5


KeyboardInterrupt: 

# DATA AUGMENTATION

In [ ]:
# Filtra solo immagini di classe 1
class_1_data = [item for item in data if "Infiltration" in item["labels"]]
class_1_paths = [item['image_path'] for item in class_1_data]
class_1_labels = [1] * len(class_1_data)

class_1_dataset = tf.data.Dataset.from_tensor_slices((class_1_paths, class_1_labels))

# Funzione di augmentation
def augment_image(image):
    # Operazioni di augmentazione
    image = tf.image.random_flip_left_right(image)  # Flip orizzontale
    image = tf.image.random_flip_up_down(image)    # Flip verticale
    image = tf.image.rot90(image, k=np.random.randint(1, 4))  # Rotazione casuale
    return image

# Funzione di preprocessamento e augmentation
def preprocess_and_augment(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224]) / 255.0
    image = augment_image(image)  # Applica augmentation
    return image, label

# Dataset augmentato per la classe 1 (ogni immagine originale produce una immagine augmentata)
augmented_class_1_dataset = class_1_dataset.map(preprocess_and_augment, num_parallel_calls=tf.data.AUTOTUNE)
# Ripeti per generare il triplo degli esempi della classe 1
augmented_class_1_dataset = augmented_class_1_dataset.repeat(2).take(num_class_1 * 2)  # 2x in più rispetto al raddoppio
print(len(augmented_class_1_dataset))
print(len(dataset_nobatch))
# Combina immagini originali e augmentate della classe 1
combined_class_1_dataset = dataset_nobatch.concatenate(augmented_class_1_dataset)
print(len(combined_class_1_dataset))

1934
5606
7540


In [ ]:
class_counts = {"class_0": 0, "class_1": 0}
for _, label in combined_class_1_dataset:
    if label.numpy() == 0:
        class_counts["class_0"] += 1
    else:
        class_counts["class_1"] += 1

print(f"Distribuzione delle classi: {class_counts}")

Distribuzione delle classi: {'class_0': 4639, 'class_1': 1934}


In [ ]:
# Applica il batching al dataset combinato
combined_class_1_dataset = combined_class_1_dataset.shuffle(buffer_size=1000)
combined_class_1_dataset = combined_class_1_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

# TensorFlow Keras  
#### (with new dataset)

In [ ]:
dataset_size = len(combined_class_1_dataset)
test_size = int(dataset_size * 0.2)
validation_size = int(dataset_size * 0.1)
train_size = dataset_size - test_size - validation_size

test_set = combined_class_1_dataset.take(test_size)  # Primi 20% per il test
validation_set = combined_class_1_dataset.skip(test_size).take(validation_size)  # Successivi 20% per validazione
train_set = combined_class_1_dataset.skip(test_size + validation_size)  # Resto per il training

In [ ]:
print(f"Dataset totale: {dataset_size}")
print(f"Test set: {test_size}")
print(f"Validation set: {validation_size}")
print(f"Training set: {train_size}")

Dataset totale: 118
Test set: 23
Validation set: 11
Training set: 84


In [ ]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_set, epochs=5, validation_data=validation_set)

Epoch 1/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 1890s 12s/step - accuracy: 0.7462 - loss: 0.7513 - val_accuracy: 0.8196 - val_loss: 0.5595
Epoch 2/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 193s 2s/step - accuracy: 0.7805 - loss: 0.6258 - val_accuracy: 0.4830 - val_loss: 1.1021
Epoch 3/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.7719 - loss: 0.6405 - val_accuracy: 0.8281 - val_loss: 0.4451
Epoch 4/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.8085 - loss: 0.5099 - val_accuracy: 0.6932 - val_loss: 0.6391
Epoch 5/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 193s 2s/step - accuracy: 0.8068 - loss: 0.5307 - val_accuracy: 0.7798 - val_loss: 0.5077


In [ ]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 37s 992ms/step - accuracy: 0.7957 - loss: 0.5079
Test Loss: 0.4851534366607666, Test Accuracy: 0.8023098111152649


In [ ]:
y_true = []  # Ground truth
y_pred = []  # Predizioni
for images, labels in test_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype(int).flatten())

print(classification_report(y_true, y_pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      1275
          

In [ ]:
y_true = []  # Ground truth
y_pred = []  # Predizioni

# Itera su tutti i batch del test set
for images, labels in test_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())  # Aggiungi tutte le etichette vere
    y_pred.extend(preds.flatten())  # Aggiungi tutte le predizioni (probabilità)

# Converti y_true e y_pred in array NumPy
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Testa diverse soglie
thresholds = [0.3, 0.5, 0.7]
for threshold in thresholds:
    y_pred_class = (y_pred > threshold).astype(int)  # Applica la soglia
    print(f"Classification report for threshold = {threshold}")
    print(classification_report(y_true, y_pred_class))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Classification report for threshold = 0.3
              precision    recall  f1-score   support

           0       0.89      